In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from os.path import join as oj
import sys
sys.path.append('../vision_fit')
sys.path.append('../vision_analyze')
import viz_weights
import numpy as np
from copy import deepcopy
import pickle as pkl
from torch.optim.lr_scheduler import StepLR
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt
import models
from dim_reduction import *
from sklearn.decomposition import MiniBatchDictionaryLearning
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
# load dset
root = oj('/scratch/users/vision/yu_dl/raaz.rsk/data', 'cifar10')
trans = transforms.Compose([transforms.ToTensor()])
test_set = dset.CIFAR10(root=root, train=False, download=True)
X_test = test_set.test_data
Y_test = np.array(test_set.test_labels)
lab_dict = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

# filter by a class
idxs = Y_test==1
X = X_test[idxs]
Y = Y_test[idxs]

# look at an image or 2
# num = 0
# plt.imshow(X[num], interpolation=None)
# plt.title(lab_dict[Y[num]])
# plt.show()

X_d = X.reshape(X.shape[0], -1)
print(X_d.shape)

Files already downloaded and verified
(1000, 3072)


In [ ]:
alpha = 1.0
n_iter = 1
n_components = 256
batch_size = 100

dico = MiniBatchDictionaryLearning(n_components=n_components, alpha=alpha, n_iter=n_iter, n_jobs=1, batch_size=batch_size) 
for i in tqdm(range(50000)):
    V = dico.fit(X_d)
    if i % 100 == 0:
        np.save('sparse_cifar/bases_iters=' + str(i) + '_alpha=' + str(alpha) + '_ncomps=' + str(n_components) + '.npy', V.components_)        
#         viz_weights.plot_weights(V.components_, dset='cifar10')